In [2]:
import os, datetime, sys
import dash
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import cv2
from pdf2image import convert_from_path
import numpy as np
import random
import torch
from sklearn.cluster import KMeans
sys.path.append("..")
#from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from skimage.transform import hough_line, hough_line_peaks

In [3]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import sklearn as sklearn
from tqdm import tqdm
import re
import plotly.express as px
from sklearn import preprocessing
from dash import Dash, dcc, html, Input, Output,dash_table
import csv

In [5]:
import time
import plotly.graph_objects as go

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

The code block aim at creating a 3D representation of the floors and their equipment. The 3D is a scatter plot put into a 3D format, following the Axis coordinates (A01Q01...)
To improve the 3D reprensentation, the same logic could be applied to the pixels coordinates - It would solve the Facade and Floor division and merge it on the same view, improving the rooms that are on the same rooms.

In [ ]:
#Dash Official dashboard version
#Wokshop to get to the desired version
#Lost the most develop version of the script; The filter related part. -> The exemple of this code block is 160125_presentation.html
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
df = merged_df
image_path = #YOUR_IMAGE
asst_path = os.getcwd()
# Replace with your image filename
image_filename = image_path  


fig = px.scatter_3d(
    merged_df,
    x="Axis_V",
    y="Axis_H",
    z="Position in the room",
    color="AKS full",
    hover_data=["Full name ",'Supplier name ']
)
fig.update_layout(
        legend_title_text = '<b>Equipment name</b>',
            scene = dict(
                xaxis_title=dict(text=f'Horizontal'),
                yaxis_title=dict(text=f'Vertical'),
                xaxis_autorange="reversed",
#                    yaxis_autorange="reversed", I think it should stay without the setting, but I may be wrong.
                xaxis = dict(
                     backgroundcolor="rgb(200, 200, 200)",
                     gridcolor="white",
                     showbackground=True,
                     showticklabels=False,
                     range = [merged_df['Axis_V'].min(),merged_df['Axis_V'].max()],
                     zerolinecolor="white",),
                yaxis = dict(
                    backgroundcolor="rgb(125, 125,125)",
                    gridcolor="white",
                    showbackground=True,
                    showticklabels=False,
                    range = [merged_df['Axis_H'].min(),merged_df['Axis_H'].max()],
                    zerolinecolor="white",),
                zaxis = dict(
                    backgroundcolor="rgb(50, 50,50)",
                    gridcolor="white",
                    showbackground=True,
                    zerolinecolor="white",
                    showticklabels=False,
                    range = [0, 9]),),
                #width=1800,
                #height=750,
                margin=dict(
                r=10, l=10,
                b=25, t=50),
                  font=dict(
                      size=16))


app = Dash(__name__, external_stylesheets=external_stylesheets, assets_folder=asst_path)
image_src = app.get_asset_url(image_filename)
app.layout = html.Div(children=[
    html.Div([
    # All elements from the top of the page
        html.Div([
                dcc.Dropdown(
                    id='dropdown-floor',
                    placeholder='Select a floor',
                    options=[{'label': cat, 'value': cat} for cat in merged_df['Level'].unique()],
                    searchable=True,
                    style={'display': 'inline-block', 'width': '65%', 'vertical-align': 'top'},
                    value=0,
                ),
                dcc.Dropdown(
                    id='dropdown-room',
                    placeholder='Select a room',
                    options=[{'label': cat, 'value': cat} for cat in merged_df['Room_Index'].unique()],
                    searchable=True,
                    style={'display': 'inline-block', 'width': '65%', 'vertical-align': 'top'},
                    value=0,
                ),
    
                dcc.Dropdown(
                    id='dropdown-pos',
                    placeholder='Select a room vertical position',
                    options=[{'label': cat, 'value': cat} for cat in merged_df['room_zone'].unique()],
                    searchable=True,
                    style={'display': 'inline-block', 'width': '65%', 'vertical-align': 'top'},
                    multi=True,
                ),
    
                dcc.Dropdown(
                    id='dropdown-cat',
                    placeholder='Select a category',
                    options=[{'label': cat, 'value': cat} for cat in merged_df['Equipment Category'].unique()],
                    searchable=True,
                    style={'display': 'inline-block', 'width': '65%', 'vertical-align': 'top'},
                    multi=True,
                ),
            ], className='four columns'),
        html.Div([
                dcc.Textarea(
                    id='textarea-example',
                    value='Welcome to the 3D Modeling of ECBs rooms and Equipment.\nGet started by selecting the room you want an overview of.\nMake the necessary modification straight on table underneath.\nHide the unecessary informations by either using the toggle button top left or with the column headers.\nDo not forget to export the new version and to upload it to Planon.\n\nUse this space to take notes.',
                    style={'display': 'inline-block','width': '100%', 'height': 175},
                ),
                html.Div(id='textarea-example-output', style={'whiteSpace': 'pre-line'}),
            ],className='three columns'),
    ], className='row'),
    
    html.Br(),

     html.Div([
        html.Div([
            dcc.Graph(
                id='graph',
                figure=fig
            ),
        ], className='seven columns'),
    
        html.Div([
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Img(
                id='image',
                style={'height': '300px', 'width': '600px'},
                src=image_src
            ),
        ], className='five columns'),
    ], className='row'),

# New Div for all elements in the new 'row' of the page
html.Div([
    html.Br(),
    dash_table.DataTable(
        id='table-equip',
        data=df.to_dict('records'),
        columns=[{"name": i, "id": i,"hideable": True,"renamable": True} for i in df.columns],
        export_format='csv',
        export_headers='display',
        merge_duplicate_headers=True,
        editable=True,
        page_size=6,
        style_filter={'backgroundColor': 'rgb(50, 50,50)'}, 
        style_header={'backgroundColor': 'rgb(50, 50,50)'},
        style_cell={'backgroundColor': 'rgb(50, 50,50)', 'color': 'white'},
        css=[
            {'selector': 'table', 'rule': 'borderCollapse: separate;'},
            {'selector': 'table', 'rule': 'border-spacing: 0 50px;'},
            {'selector': 'table', 'rule': 'margin-top: 5px;'},
            {'selector': 'td, th', 'rule': 'padding: 20px;'}
        ]
    ),
], className='row'),
])


#####
##### Initial Callback, will in function of the Floor selected, give an option of rooms
#####


@app.callback(
    Output('dropdown-room', 'options'),
    Input('dropdown-floor', 'value')
)
def update_room_options(selected_floor):
    if selected_floor is None:
        return []
    filtered_df = merged_df[merged_df['Level'] == selected_floor]
    options = [{'label': cat, 'value': cat} for cat in filtered_df['Room_Index'].unique()]
    return options


#####
##### Original Callback; Select the image, 3d figure and table in function of the Room
#####

@app.callback(
    Output('table-equip', 'data'),
     Output('graph', 'figure'),
     Output('image', 'src'),
    Input('dropdown-floor', 'value'),
    Input('dropdown-room', 'value'),
    Input('dropdown-pos', 'value'),
    Input('dropdown-cat','value')
)
def update_content(selected_floor, selected_room,selected_pos,selected_cat):
    filtered_df = merged_df[merged_df['Level']==selected_floor]
    filtered_df = filtered_df[filtered_df['Room_Index'] == selected_room]
    if bool(selected_pos):
        filtered_df = filtered_df[filtered_df['room_zone'].isin(selected_pos)]
    if bool(selected_cat):
        filtered_df = filtered_df[filtered_df['Equipment Category'].isin(selected_cat)]

    image_src = app.get_asset_url(f'HS_04_90D_CROPPED_CONTOURS_FILLED_GREYED_room_{selected_room}.0.png')
    # Update the 3D scatter plot
    fig = px.scatter_3d(
    filtered_df,
    x="Axis_V",
    y="Axis_H",
    z="Position in the room",
    symbol=filtered_df.index,
    color="Equipment Category", 
    color_discrete_sequence=px.colors.qualitative.Pastel2,
    hover_data=["Full name ",'Supplier name ','Plan_Axis_Coordinates']
    )
    
    fig.update_layout(
            legend_title_text = '<b>Equipment Category</b>',
            legend=dict(x=-0.2),
                scene = dict(
                    xaxis_title=dict(text=f'Horizontal'),
                    yaxis_title=dict(text=f'Vertical'),
                    xaxis_autorange="reversed",
#                    yaxis_autorange="reversed", I think it should stay without the setting, but I may be wrong.
                    xaxis = dict(
                         backgroundcolor="rgb(200, 200, 200)",
                         gridcolor="white",
                         showbackground=True,
                         showticklabels=False,
                         range = [filtered_df['Axis_V'].min(),filtered_df['Axis_V'].max()],
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="rgb(125, 125,125)",
                        gridcolor="white",
                        showbackground=True,
                        showticklabels=False,
                        range = [filtered_df['Axis_H'].min(),filtered_df['Axis_H'].max()],
                        zerolinecolor="white",),
                    zaxis = dict(
                        backgroundcolor="rgb(50, 50,50)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white",
                        showticklabels=False,
                        range = [0, 9]),),
                    #width=1800,
                    #height=750,
                    margin=dict(
                    r=10, l=10,
                    b=25, t=50),
                      font=dict(
                            size=12,
                        )
    )
    if len(filtered_df) != 0:
        i = filtered_df[filtered_df['Room_Index'] == selected_room]['Horizontal'].unique()[0]
        j = filtered_df[filtered_df['Room_Index'] == selected_room]['Vertical'].unique()[0]

        fig.update_layout(
                    scene = dict(
                        xaxis_title=dict(text=f'Vertical axis {j}'),
                        yaxis_title=dict(text=f'Horizontal axis {i}')))
        
    return filtered_df.to_dict('records'), fig, image_src





if __name__ == "__main__":
    app.run_server(debug=False)